In [ ]:
import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt

# Load Configuration

In [ ]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
ds_ref = xr.open_zarr(CONFIG["path_ec_euro"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))
ds_mod = xr.open_zarr(CONFIG["path_xgb_v1"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))

# Example time series

In [ ]:
fname = f"/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/spatial/bias.zarr"
ds_bias = xr.open_zarr(fname)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

vmin = np.nanpercentile(ds_bias.sel(variable="stl1").data, 1, axis=0)
vmax = np.nanpercentile(ds_bias.sel(variable="stl1").data, 99, axis=0)
im = ax.scatter(ds_bias["lon"], ds_bias["lat"], c=ds_bias.sel(variable="stl1").data, s=10, vmin=vmin, vmax=vmax)
fig.colorbar(im)

plt.show()

In [ ]:
i = ds_bias.sel(variable="swvl1").data.argmax(dim="x").values

In [ ]:
plt.plot(ds_ref.sel(variable="swvl1").data.isel(x=i), label="ref")
plt.plot(ds_mod.sel(variable="swvl1").data.isel(x=i), label="mod")
plt.legend()
plt.show()

# Power Spectra

Compute the power spectra of all variables conatined in both data sets. 

In [ ]:
# Path for saving the plots:
path_png = "/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/visualization/spectrum"

# Loop through all variables contained in both data sets:
common_vars = np.intersect1d(ds_ref.variable, ds_mod.variable)
for var in common_vars:
    vis.power_spectrum(ds_mod, ds_ref, var, path_png)

# Condition on Season

In [ ]:
season_masks = {"DJF": ds_ref["time"].dt.month.isin([12,1,2]), 
                "MAM": ds_ref["time"].dt.month.isin([3,4,5]),
                "JJA": ds_ref["time"].dt.month.isin([6,7,8]), 
                "SON": ds_ref["time"].dt.month.isin([9,10,11])}

In [ ]:
#common_vars = ["stl1"] 
common_vars = np.intersect1d(ds_mod.variable, ds_ref.variable)

for var in common_vars:
    seasonal_results = np.full([4, ds_ref.sizes["x"]], np.nan)

    for i, sm in enumerate(season_masks.values()):
        seasonal_results[i] = stm.bias(ds_mod.isel(time=sm), ds_ref.isel(time=sm), vars=var)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.boxplot(seasonal_results.T, labels=season_masks.keys())
    ax.set_ylim(np.nanpercentile(seasonal_results, 0.05), np.nanpercentile(seasonal_results, 99.95))
    ax.set(title=f"{var} bias in different seasons")

    plt.show()

In [ ]:

ii = np.random.randint(low=0, high=ds_ref.sizes["x"], size=20)

for i in ii:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    #ax.plot(ds_ref.rolling(time=7).mean()["time"], ds_ref.sel(variable=var).data.isel(x=i).rolling(time=7).mean(), label="ref")
    #ax.plot(ds_mod.rolling(time=7).mean()["time"], ds_mod.sel(variable=var).data.isel(x=i).rolling(time=7).mean(), label="mod")
    err = (ds_mod.sel(variable=var).data.isel(x=i) - ds_ref.sel(variable=var).data.isel(x=i))#.rolling(time=7).mean()
    ax.plot(err["time"], err)
    #plt.legend()
    plt.show()

    for key in season_masks:
        season_bias = stm.bias(ds_mod.isel(time=season_masks[key], x=i), ds_ref.isel(time=season_masks[key], x=i), vars=var).values
        print(f"{key}\t{season_bias:.2f}")

# Condition on time

In [ ]:
standard_time = ds_ref["time"].dt.hour.expand_dims(dim={"x": ds_ref["x"]})

local_time = standard_time - (4 * ds_ref["lon"])/60. #the sun takes 4 min to traverse 1° longitude
local_time = local_time.T % 24 #convert negative values

day_mask = (local_time >= 6) & (local_time < 18)

In [ ]:
#common_vars = ["stl1"] 
common_vars = np.intersect1d(ds_mod.variable, ds_ref.variable)

for var in common_vars:
    diurnal_results = np.full([2, ds_ref.sizes["x"]], np.nan)

    diurnal_results[0] = stm.bias(ds_mod.where(day_mask), ds_ref.where(day_mask), vars=var)
    diurnal_results[1] = stm.bias(ds_mod.where(~day_mask), ds_ref.where(~day_mask), vars=var)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.boxplot(diurnal_results.T, labels=["day","night"])
    ax.set_ylim(np.nanpercentile(diurnal_results, 0.05), np.nanpercentile(diurnal_results, 99.95))
    ax.set(title=f"{var} bias in different times of day")

    plt.show()